<a href="https://www.kaggle.com/code/vovanquangnbk/drowsy-train-eyelandmarks?scriptVersionId=144340922" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Setup

In [ ]:
!pip -q install mediapipe
!pip -q install torchsummary

In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
import itertools
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import mediapipe as mp

In [ ]:
CFG = {
    'data_dir': '/kaggle/input/drowsy-eye-keypoints',
    'seed': 719,
    'model_arch': 'CNN',
    'train_all': False,
    'epochs': 5,
    'used_epochs':[2,3,4],
    'train_bs': 8,
    'valid_bs': 8,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 2,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'show_examples': True,
}

In [ ]:
paths = os.path.join(CFG['data_dir'], "*.csv")
df = pd.concat(map(pd.read_csv, glob(paths)))
df = df.sample(frac=1)
df = df.reset_index(drop=True)

train = df[df['fold'] != 'fold1']
test = df[df['fold'] == 'fold1']

if CFG['show_examples']:
    print(train.shape)
    print(test.shape)
    print(test.head())

## Utils

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## Dataset

In [ ]:
# Eye Landmark indices
mp_face_mesh = mp.solutions.face_mesh

LEFT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
RIGHT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))             


def indexes(list_obj, element_list):
    return [list_obj.index(e) for e in element_list]

LEFT_EYE_UPPER = indexes(LEFT_EYE_INDEXES, [398, 384, 385, 386, 387, 388, 466])
LEFT_EYE_LOWER = indexes(LEFT_EYE_INDEXES, [382, 381, 380, 374, 373, 390, 249])
LEFT_EYE_LEFT = LEFT_EYE_INDEXES.index(362)
LEFT_EYE_RIGHT = LEFT_EYE_INDEXES.index(263)
if CFG['show_examples']:
    print(LEFT_EYE_UPPER)
    print(LEFT_EYE_LOWER)
    print(LEFT_EYE_LEFT)
    print(LEFT_EYE_RIGHT)

RIGHT_EYE_UPPER = indexes(RIGHT_EYE_INDEXES, [246, 161, 160, 159, 158, 157, 173])
RIGHT_EYE_LOWER = indexes(RIGHT_EYE_INDEXES, [7, 163, 144, 145, 153, 154, 155])
RIGHT_EYE_LEFT = RIGHT_EYE_INDEXES.index(33)
RIGHT_EYE_RIGHT = RIGHT_EYE_INDEXES.index(133)
if CFG['show_examples']:
    print(LEFT_EYE_UPPER)
    print(LEFT_EYE_LOWER)
    print(LEFT_EYE_LEFT)
    print(LEFT_EYE_RIGHT)

# Source: https://raw.githubusercontent.com/tensorflow/tfjs-models/master/face-landmarks-detection/mesh_map.jpg

In [ ]:
class MyDataset(Dataset):
    def __init__(self,
                 df,
                 data_root=None,
                 transforms=None,
                 output_label=True,
                 one_hot_label=False,
                ):

        super().__init__()
        self.df = df.copy()
        self.data_root = data_root
        self.transforms = transforms
        self.output_label = output_label
        self.one_hot_label = one_hot_label

        if output_label == True:
            self.labels = self.df['label'].values
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):

        # get labels
        if self.output_label:
            label = self.labels[index]

        x_dir = os.path.join(
            self.data_root, 
            self.df.iloc[index]['fold'],
            self.df.iloc[index]['id'])
        x = np.load(x_dir)
        upper = x[:,LEFT_EYE_UPPER,:][:,:,[1,3]]
        lower = x[:,LEFT_EYE_LOWER,:][:,:,[1,3]]
        eye_corner = x[:,LEFT_EYE_LEFT, [1,3]] - x[:,LEFT_EYE_RIGHT, [1,3]]
        
        EAR = np.sum(np.abs(upper - lower), axis=1) / (7*np.abs(eye_corner) + 1e-10)
        
        if EAR.shape[0] >= 100:
            EAR = EAR[:100,:]
        else:
            length = EAR.shape[0]
            EAR = np.pad(EAR, ((0,100-length),(0,0)), mode='constant', constant_values=-1)
            
        EAR = torch.from_numpy(EAR)
        EAR = EAR.permute(1,0)
        EAR = F.normalize(EAR)
        
        if self.output_label == True:
            return EAR, label
        else:
            return EAR

# Test dataset
if CFG['show_examples']:
    dataset = MyDataset(train, CFG['data_dir'])
    for i, (EAR, label) in enumerate(dataset):
        print(EAR.shape, label, torch.isnan(EAR).sum())
        if i > 10:
            break

In [ ]:
def prepare_dataloader(df, trn_idx, val_idx, train_all=CFG['train_all']):

    from catalyst.data.sampler import BalanceClassSampler

    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)

    train_ds = MyDataset(train_, data_root=CFG['data_dir'], output_label=True, one_hot_label=False)
    valid_ds = MyDataset(valid_, data_root=CFG['data_dir'], output_label=True)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,
        num_workers=CFG['num_workers'],
#         sampler=BalanceClassSampler(labels=train_['label'].values, mode="downsampling")
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

# Test data loader
if CFG['show_examples']:
    dataset = MyDataset(train, CFG['data_dir'])
    trn_idx = train[train['fold'] == 'fold4'].index.tolist()
    val_idx = train[train['fold'] == 'fold3'].index.tolist()
    train_loader, val_loader = prepare_dataloader(train, trn_idx, val_idx, train_all=CFG['train_all'])

    print(len(train_loader))
    for i, (x, label) in enumerate(train_loader):
        x = x.to(CFG['device'])
        label = label.to(CFG['device'])
        print(x.shape, label.shape)
        if i > 5:
            break
    print(len(val_loader))
    for i, (x, label) in enumerate(val_loader):
        x = x.to(CFG['device'])
        label = label.to(CFG['device'])
        print(x.shape, label.shape)
        if i > 5:
            break

## Model

In [ ]:
class MyClassifier(nn.Module):
    def __init__(self, num_classes=2, out_channels=[20,20]):
        super(MyClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=out_channels[0],  kernel_size = 10)
        self.conv2 = nn.Conv1d(in_channels=out_channels[0], out_channels=out_channels[1], kernel_size=5)
        self.pool = nn.MaxPool1d(2)
        self.batchnorm1 = nn.BatchNorm1d(out_channels[0])
        self.batchnorm2 = nn.BatchNorm1d(out_channels[1])
        self.drop = nn.Dropout1d(p=0.2)
        self.fc = nn.Linear(20*20, out_features=num_classes)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x))) 
        x = self.batchnorm1(x)
        x = F.relu(self.pool(self.conv2(x)))
        x = self.batchnorm2(x)
        x = F.dropout(self.drop(x), training=self.training)
        x = x.view(-1, 20*20)
        x = self.fc(x)
        return x

In [ ]:
from torchsummary import summary
model = MyClassifier()
summary(model, (2, 100))

## Traning API

In [ ]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scaler, scheduler=None, schd_batch_update=False, threshold=0.5):
    model.train()

    t = time.time()
    running_loss = None
    preds_all = []
    y_all = []
    threshold = threshold

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (X, y) in pbar:
        X = X.to(device).float()
        y = y.to(device).long()

        with autocast():
            preds = model(X)
            preds_all += [torch.argmax(preds, 1).detach().cpu().numpy()]
            y_all += [y.detach().cpu().numpy()]

            loss = loss_fn(preds, y)

            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'

                pbar.set_description(description)

    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    
    preds_all = np.concatenate(preds_all)
    y_all = np.concatenate(y_all)
    print('train multi-class accuracy = {:.4f}'.format((preds_all==y_all).mean()))

def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False, threshold=0.5):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    preds_all = []
    y_all = []

    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (X, y) in pbar:
        X = X.to(device).float()
        y = y.to(device).long()

        preds = model(X)
        preds_all += [torch.argmax(preds, 1).detach().cpu().numpy()]
        y_all += [y.detach().cpu().numpy()]

        loss = loss_fn(preds, y)

        loss_sum += loss.item()*y.shape[0]
        sample_num += y.shape[0]

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)

    preds_all = np.concatenate(preds_all)
    y_all = np.concatenate(y_all)
    print('validation multi-class accuracy = {:.4f}'.format((preds_all==y_all).mean()))

    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
            
def inference_one_epoch(model, data_loader, device):
    model.eval()
    preds_all = []

    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, X in pbar:
        X = X.to(device).float()

        preds = model(X)
        preds_all += [torch.softmax(preds, 1).detach().cpu().numpy()]

    preds_all = np.concatenate(preds_all)
    
    return preds_all

## Main

In [ ]:
if __name__ == '__main__':
    seed_everything(CFG['seed'])
    folds = []
    
    trn_idx = df[df['fold'] != f"fold1"].index.tolist()
    test_idx = df[df['fold'] == f"fold1"].index.tolist()

    print(len(trn_idx), len(test_idx))

    train_loader, val_loader = prepare_dataloader(df, trn_idx, test_idx)

    model = MyClassifier().to(CFG['device'])
    scaler = GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)

    loss_fn = nn.CrossEntropyLoss().to(CFG['device'])

    for epoch in range(CFG['epochs']):
        print("\n")
        train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, CFG['device'], scaler, scheduler=scheduler, schd_batch_update=False)

        with torch.no_grad():
            valid_one_epoch(epoch, model, loss_fn, val_loader, CFG['device'], scheduler=None, schd_loss_update=False, threshold=0.5)
        if epoch >= 2:
            torch.save(model.state_dict(), '{}_{}'.format(CFG['model_arch'], epoch))

    del model, optimizer, train_loader, val_loader, scaler, scheduler
    torch.cuda.empty_cache()

## Evaluation

In [ ]:
best_model = MyClassifier().to(CFG['device'])
tst_preds_all = []
threshold = 0.085

test_ds = MyDataset(test, data_root=CFG['data_dir'], output_label=False)
tst_loader = torch.utils.data.DataLoader(
    test_ds,
    batch_size=CFG['valid_bs'],
    num_workers=CFG['num_workers'],
    shuffle=False,
    pin_memory=False,
)

tst_preds = []

start_time = time.time()
for i, epoch in enumerate(CFG['used_epochs']):
    best_model.load_state_dict(torch.load('{}_{}'.format(CFG['model_arch'], epoch), map_location=torch.device(CFG['device'])))

    with torch.no_grad():
        tst_preds += [inference_one_epoch(best_model, tst_loader, CFG['device'])]

tst_preds_all += [np.mean(tst_preds, axis=0)]

del best_model
torch.cuda.empty_cache()

tst_preds_all = np.mean(tst_preds_all, axis=0)
tst_preds_all = np.argmax(tst_preds_all, axis=1)

test['preds'] = tst_preds_all
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

print("Multi-class accuracy: ", accuracy_score(test["label"], test["preds"]))
print("F1-score: ", f1_score(test["label"], test["preds"]))
print("Precision: ", precision_score(test["label"], test["preds"]))
print("Recall: ", recall_score(test["label"], test["preds"]))

In [ ]:
import seaborn as sns
y_true = test["label"]
y_pred = test["preds"]
cf_mt = confusion_matrix(y_true, y_pred)
sns.heatmap(cf_mt, annot=True)